<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_designite_guava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checa ambiente

In [1]:
!java --version

openjdk 11.0.20 2023-07-18
OpenJDK Runtime Environment (build 11.0.20+8-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.20+8-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [2]:
!python3 --version

Python 3.10.12


In [3]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (114 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 120831 files and directories currently install

# Funcões

In [4]:
import pandas as pd
import os
import requests

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try:
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd guava && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta nome_repositorio
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio nome_repositorio
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta nome_repositorio
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto nome_repositorio)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto nome_repositorio
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto nome_repositorio
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do nome_repositorio
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1])
      lista_comment.append(each[2])
      lista_code.append(each[3])
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp,
    'files':lista_files,
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

In [5]:
def download_designite():
  try:
    url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Excetpion as e:
    print(f'Erro: {str(e)}')

def run_designite(tag, pasta_testes):
  try:
    os.environ['TESTES'] = pasta_testes
    os.environ['GUAVA'] = tag + 'guava/src'
    cria_pasta_testes = !mkdir $TESTES
    print(cria_pasta_testes)
    executa_designite = !java -jar DesigniteJava.jar -i $GUAVA -o $TESTES
    print(executa_designite)
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

In [6]:
def is_valid_url(url):
    try:
        response = requests.head(url)
        return response.status_code in range(200, 300)  # Check if status code is in the 2xx range
    except requests.exceptions.RequestException:
        return False  # Any error or exception indicates an invalid or non-existent URL

'''
Example usage:
url1 = "https://www.example.com"
url2 = "https://www.invalid-url.com"
print(is_valid_url(url1))  # Output: True
print(is_valid_url(url2))  # Output: False
'''

'\nExample usage:\nurl1 = "https://www.example.com"\nurl2 = "https://www.invalid-url.com"\nprint(is_valid_url(url1))  # Output: True\nprint(is_valid_url(url2))  # Output: False\n'

In [7]:
import re

def get_elements_with_pattern(strings_list, pattern):
    matching_elements = [s for s in strings_list if re.search(pattern, s)]
    return matching_elements

# Example list of strings
strings_list = ['gson-parent-1.0', 'gson-parent-2.0', 'gson-child-1.0', 'gson-parent-3.0']
pattern_to_match = r'gson-parent'

result = get_elements_with_pattern(strings_list, pattern_to_match)
print(result)

['gson-parent-1.0', 'gson-parent-2.0', 'gson-parent-3.0']


# Análise Designite

In [8]:
import requests

url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
response = requests.get(url)

with open(filename, 'wb') as f:
    f.write(response.content)

print(f'{filename} downloaded successfully!')

DesigniteJava.jar downloaded successfully!


In [9]:
!git clone https://github.com/google/guava.git

Cloning into 'guava'...
remote: Enumerating objects: 392311, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 392311 (delta 177), reused 308 (delta 163), pack-reused 391975
Receiving objects: 100% (392311/392311), 428.47 MiB | 24.35 MiB/s, done.
Resolving deltas: 100% (290482/290482), done.


In [10]:
!tree guava

guava
├── android
│   ├── guava
│   │   ├── javadoc-link
│   │   │   ├── checker-framework
│   │   │   │   └── package-list
│   │   │   ├── j2objc-annotations
│   │   │   │   └── package-list
│   │   │   └── jsr305
│   │   │       └── package-list
│   │   ├── pom.xml
│   │   └── src
│   │       └── com
│   │           └── google
│   │               ├── common
│   │               │   ├── annotations
│   │               │   │   ├── Beta.java
│   │               │   │   ├── GwtCompatible.java
│   │               │   │   ├── GwtIncompatible.java
│   │               │   │   ├── J2ktIncompatible.java
│   │               │   │   ├── package-info.java
│   │               │   │   └── VisibleForTesting.java
│   │               │   ├── base
│   │               │   │   ├── Absent.java
│   │               │   │   ├── AbstractIterator.java
│   │               │   │   ├── Ascii.java
│   │               │   │   ├── CaseFormat.java
│   │               │   │   ├── CharMatcher.java
│   │               │ 

### Pega a lista de tags

In [13]:
lista_tags_guava = !cd guava && git tag
lista_tags_guava

['failureaccess-v1.0.1',
 'jdk5-backport-base',
 'jdk5-backport-branch-point',
 'jdk5-backport-release',
 'jdk5-backport-v16.0',
 'jdk5-backport-v16.0-rc1',
 'jdk5-backport-v17.0',
 'jdk5-backport-v17.0-compatibility',
 'jdk5-backport-v17.0-post',
 'jdk5-backport-v17.0-rc1',
 'jdk5-backport-v17.0-rc2',
 'release',
 'released-all-futures',
 'v1.0',
 'v10.0',
 'v10.0-rc1',
 'v10.0-rc2',
 'v10.0-rc3',
 'v10.0.1',
 'v11.0',
 'v11.0-rc1',
 'v11.0.1',
 'v11.0.2',
 'v12.0',
 'v12.0-rc1',
 'v12.0-rc2',
 'v12.0.1',
 'v13.0',
 'v13.0-final',
 'v13.0-rc1',
 'v13.0-rc2',
 'v13.0.1',
 'v14.0',
 'v14.0-rc1',
 'v14.0-rc2',
 'v14.0-rc3',
 'v14.0.1',
 'v15.0',
 'v15.0-cdi1.0',
 'v15.0-rc1',
 'v16.0',
 'v16.0-rc1',
 'v16.0.1',
 'v17.0',
 'v17.0-rc1',
 'v17.0-rc2',
 'v18.0',
 'v18.0-rc1',
 'v18.0-rc2',
 'v19.0',
 'v19.0-rc1',
 'v19.0-rc2',
 'v19.0-rc3',
 'v2.0',
 'v20.0',
 'v20.0-rc1',
 'v21.0',
 'v21.0-rc1',
 'v21.0-rc2',
 'v22.0',
 'v22.0-rc1',
 'v23.0',
 'v23.0-rc1',
 'v23.1',
 'v23.2',
 'v23.3',
 'v2

In [14]:
pattern_to_extract = r'v28'
get_elements_with_pattern(lista_tags_guava, pattern_to_extract)

['v28.0', 'v28.1', 'v28.2']

In [15]:
pattern_to_extract = r'v32'
get_elements_with_pattern(lista_tags_guava, pattern_to_extract)

['v32.0.0', 'v32.0.1', 'v32.1.0', 'v32.1.1', 'v32.1.2']

# Análise da tag guava_28

In [16]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_28 = get_commit_tag(tag='v28.0')

In [17]:
cria_pasta_n_commit(repositorio='guava', pasta_n_commit='guava_2_8')

[]

In [18]:
!ls guava_2_8

android			 guava		integration-tests	proguard
CONTRIBUTING.md		 guava-bom	javadoc-stylesheet.css	README.md
CONTRIBUTORS		 guava-gwt	LICENSE			refactorings
cycle_suppress_list.txt  guava-testlib	overview.html		util
futures			 guava-tests	pom.xml


In [19]:
checkout_n_commit('guava_2_8', commit_tag_28[0])

Checkout do repositorio no commit 9957203c956016b485696ce9d1a4d60f1019c5fd realizado com sucesso!


In [20]:
ls -liath guava_2_8/guava/src

total 12K
137866 drwxr-xr-x 4 root root 4.0K Aug 24 15:20 ../
137876 drwxr-xr-x 3 root root 4.0K Aug 24 15:20 ./
137877 drwxr-xr-x 3 root root 4.0K Aug 24 15:20 com/


In [21]:
!java -jar DesigniteJava.jar -i guava_2_8/guava/src -o guava_2_8

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due 

In [22]:
!ls -liat guava_2_8/designCodeSmells.csv

134939 -rw-r--r-- 1 root root 115812 Aug 24 15:21 guava_2_8/designCodeSmells.csv


In [23]:
df_design_smells = pd.read_csv('guava_2_8/designCodeSmells.csv')

In [24]:
df_design_smells

,Project Name,Package Name,Type Name,Code Smell
0,guava,com.google.thirdparty.publicsuffix,PublicSuffixPatterns,Deficient Encapsulation
1,guava,com.google.common.primitives,UnsignedInteger,Deficient Encapsulation
2,guava,com.google.common.primitives,UnsignedInteger,Insufficient Modularization
3,guava,com.google.common.primitives,UnsignedInts,Insufficient Modularization
4,guava,com.google.common.primitives,SignedBytes,Deficient Encapsulation
...,...,...,...,...
1599,guava,com.google.common.reflect,TypeParameter,Unutilized Abstraction
1600,guava,com.google.common.reflect,TypeParameter,Broken Hierarchy
1601,guava,com.google.common.reflect,TypeToInstanceMap,Unutilized Abstraction
1602,guava,com.google.common.reflect,TypeResolver,Insufficient Modularization


In [25]:
df_design_smells['Type Name'].unique()

array(['PublicSuffixPatterns', 'UnsignedInteger', 'UnsignedInts',
       'SignedBytes', 'Bytes', 'ByteArrayAsList', 'Floats',
       'FloatArrayAsList', 'Booleans', 'BooleanArrayAsList', 'Doubles',
       'DoubleArrayAsList', 'Shorts', 'ShortArrayAsList', 'Chars',
       'CharArrayAsList', 'ImmutableIntArray', 'Builder', 'AsList',
       'UnsignedBytes', 'LexicographicalComparatorHolder',
       'UnsignedLongs', 'Ints', 'IntArrayAsList', 'ImmutableDoubleArray',
       'UnsignedLong', 'ImmutableLongArray', 'Longs', 'LongArrayAsList',
       'Quantiles', 'Scale', 'ScaleAndIndex', 'ScaleAndIndexes',
       'LongMath', 'StatsAccumulator', 'Stats', 'LinearTransformation',
       'LinearTransformationBuilder', 'RegularLinearTransformation',
       'VerticalLinearTransformation', 'IntMath',
       'SubscriberExceptionHandler', 'Subscriber',
       'SynchronizedSubscriber', 'SubscriberRegistry', 'EventBus',
       'LoggingHandler', 'PerThreadQueuedDispatcher',
       'LegacyAsyncDispatcher', '

In [26]:
# Esta lista eh oriunda da analise AMLOC x FOC e das analises dos 3o quartis AMLOC e FOC
lista_classes_criticas = ['AbstractBiMap',
 'AbstractExecutionThreadService',
 'AbstractFuture',
 'AbstractIdleService',
 'AbstractScheduledService',
 'AggregateFuture',
 'ArrayTable',
 'BaseEncoding',
 'BloomFilter',
 'ByteStreams',
 'CacheBuilder',
 'CacheBuilderSpec',
 'CacheLoader',
 'CharStreams',
 'ClassPath',
 'ClosingFuture',
 'CollectionFuture',
 'CompactHashMap',
 'ConcurrentHashMultiset',
 'Converter',
 'Doubles',
 'ElementTypesAreNonnullByDefault',
 'Equivalence',
 'EvictingQueue',
 'ExecutionSequencer',
 'FileBackedOutputStream',
 'Files',
 'Floats',
 'FluentIterable',
 'Futures',
 'HashBiMap',
 'Hashing',
 'Helpers',
 'HttpHeaders',
 'IgnoreJRERequirement',
 'ImmutableBiMap',
 'ImmutableCollection',
 'ImmutableList',
 'ImmutableListMultimap',
 'ImmutableMap',
 'ImmutableMultimap',
 'ImmutableMultiset',
 'ImmutableRangeMap',
 'ImmutableRangeSet',
 'ImmutableSet',
 'ImmutableSetMultimap',
 'ImmutableSortedMap',
 'ImmutableSortedSet',
 'ImmutableTable',
 'InetAddresses',
 'InternetDomainName',
 'InterruptibleTask',
 'Ints',
 'Iterables',
 'Iterators',
 'Java8Compatibility',
 'JdkFutureAdapters',
 'ListenableFuture',
 'Lists',
 'LocalCache',
 'LongMath',
 'Longs',
 'MapMakerInternalMap',
 'Maps',
 'MediaType',
 'MinMaxPriorityQueue',
 'MoreExecutors',
 'MoreObjects',
 'Multimaps',
 'Multisets',
 'NullnessCasts',
 'Ordering',
 'ParametricNullness',
 'Platform',
 'PublicSuffixPatterns',
 'Range',
 'RegularImmutableMap',
 'ServiceManager',
 'Sets',
 'SimpleTimeLimiter',
 'Striped',
 'Synchronized',
 'Throwables',
 'Traverser',
 'TreeRangeSet',
 'TypeToken',
 'Types',
 'Uninterruptibles',
 'ValueGraphBuilder',
 'package-info']

In [27]:
pesquisa = df_design_smells['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [28]:
df_critical_classes = df_design_smells[pesquisa]
df_critical_classes

,Project Name,Package Name,Type Name,Code Smell
0,guava,com.google.thirdparty.publicsuffix,PublicSuffixPatterns,Deficient Encapsulation
3,guava,com.google.common.primitives,UnsignedInts,Insufficient Modularization
8,guava,com.google.common.primitives,Floats,Deficient Encapsulation
9,guava,com.google.common.primitives,Floats,Cyclic-Dependent Modularization
10,guava,com.google.common.primitives,Floats,Insufficient Modularization
...,...,...,...,...
1577,guava,com.google.common.reflect,TypeToken,Unutilized Abstraction
1578,guava,com.google.common.reflect,TypeToken,Broken Hierarchy
1579,guava,com.google.common.reflect,TypeToken,Cyclic-Dependent Modularization
1580,guava,com.google.common.reflect,TypeToken,Insufficient Modularization


In [29]:
df_critical_classes.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes = df_critical_classes.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_critical_classes

,ProjectName,PackageName,TypeName,CodeSmell
9,guava,com.google.common.primitives,Floats,Cyclic-Dependent Modularization
16,guava,com.google.common.primitives,Doubles,Cyclic-Dependent Modularization
37,guava,com.google.common.primitives,Ints,Cyclic-Dependent Modularization
49,guava,com.google.common.primitives,Longs,Cyclic-Dependent Modularization
56,guava,com.google.common.math,LongMath,Cyclic-Dependent Modularization
...,...,...,...,...
1478,guava,com.google.common.collect,ImmutableList,Cyclic-Dependent Modularization
1551,guava,com.google.common.collect,ImmutableSetMultimap,Cyclic-Dependent Modularization
1561,guava,com.google.common.reflect,ClassPath,Cyclic-Dependent Modularization
1567,guava,com.google.common.reflect,Types,Cyclic-Dependent Modularization


In [30]:
df_analise_designite_critical_classes = df_critical_classes.sort_values(by='TypeName')
df_analise_designite_critical_classes.to_csv('guava_2_8/analise_designite_classes_criticas.csv', index=False)
df_design_smells.to_csv('guava_2_8/analise_designite.csv', index=False)

In [31]:
!cd guava_2_8 && zip ds_guava_2_8.zip *.csv

  adding: analise_designite_classes_criticas.csv (deflated 89%)
  adding: analise_designite.csv (deflated 92%)
  adding: designCodeSmells.csv (deflated 92%)
  adding: implementationCodeSmells.csv (deflated 93%)
  adding: methodMetrics.csv (deflated 92%)
  adding: typeMetrics.csv (deflated 80%)


# Análise da tag v32.1.2


In [32]:
commit_tag_v32_1_2 = get_commit_tag(tag='v32.1.2')

In [33]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
cria_pasta_n_commit(repositorio='guava', pasta_n_commit='guava_32_1_2')

[]

In [34]:
checkout_n_commit('guava_32_1_2', commit_tag_v32_1_2[0])

Checkout do repositorio no commit db74bd2fdac443223d45e6fc5c66548542be1081 realizado com sucesso!


In [35]:
!java -jar DesigniteJava.jar -i guava_32_1_2/guava/src/ -o guava_32_1_2

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due 

In [36]:
df_design_smells_final = pd.read_csv('guava_32_1_2/designCodeSmells.csv')

In [37]:
df_design_smells_final

,Project Name,Package Name,Type Name,Code Smell
0,guava,com.google.thirdparty.publicsuffix,PublicSuffixPatterns,Deficient Encapsulation
1,guava,com.google.common.primitives,UnsignedInteger,Deficient Encapsulation
2,guava,com.google.common.primitives,UnsignedInteger,Insufficient Modularization
3,guava,com.google.common.primitives,UnsignedInts,Insufficient Modularization
4,guava,com.google.common.primitives,SignedBytes,Deficient Encapsulation
...,...,...,...,...
1664,guava,com.google.common.reflect,TypeParameter,Unutilized Abstraction
1665,guava,com.google.common.reflect,TypeParameter,Broken Hierarchy
1666,guava,com.google.common.reflect,TypeToInstanceMap,Unutilized Abstraction
1667,guava,com.google.common.reflect,TypeResolver,Insufficient Modularization


In [38]:
pesquisa_final = df_design_smells_final['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [39]:
df_critical_classes_final = df_design_smells_final[pesquisa_final]
df_critical_classes_final.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes_final = df_critical_classes_final.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")

In [40]:
df_critical_classes_final

,ProjectName,PackageName,TypeName,CodeSmell
13,guava,com.google.common.primitives,Floats,Cyclic-Dependent Modularization
24,guava,com.google.common.primitives,Doubles,Cyclic-Dependent Modularization
51,guava,com.google.common.primitives,Ints,Cyclic-Dependent Modularization
65,guava,com.google.common.primitives,Longs,Cyclic-Dependent Modularization
72,guava,com.google.common.math,LongMath,Cyclic-Dependent Modularization
...,...,...,...,...
1545,guava,com.google.common.collect,ImmutableList,Cyclic-Dependent Modularization
1619,guava,com.google.common.collect,ImmutableSetMultimap,Cyclic-Dependent Modularization
1627,guava,com.google.common.reflect,ClassPath,Cyclic-Dependent Modularization
1633,guava,com.google.common.reflect,Types,Cyclic-Dependent Modularization


In [41]:
df_analise_designite_critical_classes_final = df_critical_classes_final.sort_values(by='TypeName')
df_analise_designite_critical_classes_final.to_csv('guava_32_1_2/analise_designite_classes_criticas.csv', index=False)
df_design_smells_final.to_csv('guava_32_1_2/analise_designite.csv', index=False)

In [42]:
!cd guava_32_1_2 && zip ds_guava_32_1_2.zip *.csv

  adding: analise_designite_classes_criticas.csv (deflated 88%)
  adding: analise_designite.csv (deflated 92%)
  adding: designCodeSmells.csv (deflated 92%)
  adding: implementationCodeSmells.csv (deflated 94%)
  adding: methodMetrics.csv (deflated 91%)
  adding: typeMetrics.csv (deflated 80%)
